In [ ]:
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [39]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [40]:
from langchain_neo4j import Neo4jGraph ## Connect with Neo4j Graph database
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [41]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("Groq_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016399A7D940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016399A69130>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [42]:
from langchain_neo4j import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    exclude_types=["Genre"],
    verbose=True,
    allow_dangerous_requests=True  # allows updates/mutations if needed
)

In [43]:
chain.graph.schema

'Node properties:\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}\nPerson {name: STRING}\nGenre {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Movie)-[:IN_GENRE]->(:Genre)\n(:Person)-[:DIRECTED]->(:Movie)\n(:Person)-[:ACTED_IN]->(:Movie)'

In [44]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)"
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": (
            "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), "
            "(a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) "
            "WHERE g1.name = 'Comedy' AND g2.name = 'Action' "
            "RETURN DISTINCT a.name"
        )
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": (
            "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) "
            "WHERE a.name STARTS WITH 'John' "
            "WITH d, COUNT(DISTINCT a) AS JohnsCount "
            "WHERE JohnsCount >= 3 "
            "RETURN d.name"
        )
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name"
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": (
            "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) "
            "RETURN a.name, COUNT(m) AS movieCount "
            "ORDER BY movieCount DESC LIMIT 1"
        )
    }
]


In [45]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\nCypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

In [46]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}, {'question': "Which directors have made mo

In [47]:
print(prompt.format(question="How many artists are there?", schema="foo"))

You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query

User input:How many artists are there?
Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input:Which actors played in the movie Casino?
Cypher query:MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

User input:How many movies has Tom Hanks acted in?
Cypher query:MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

User input:List all the genres of the movie Schindler's List
Cypher query:MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input:Which actors have worked in movies from both the comedy and action genres?
Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input:Which directors have made movies with at least three differ

In [48]:
llm
chain=GraphCypherQAChain.from_llm(graph=graph, llm=llm, cypher_prompt=prompt, verbose=True, allow_dangerous_requests=True)

In [49]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods played in the movie Casino.  \n'}

In [50]:
chain.invoke("Identify movies where directors also played a role in the film")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) 
RETURN m.title, p.name

Full Context:
[{'m.title': 'Mighty Aphrodite', 'p.name': 'Woody Allen'}, {'m.title': 'French Twist (Gazon maudit)', 'p.name': 'Josiane Balasko'}, {'m.title': 'Heidi Fleiss: Hollywood Madam', 'p.name': 'Nick Broomfield'}, {'m.title': 'Bridges of Madison County, The', 'p.name': 'Clint Eastwood'}, {'m.title': 'If Lucy Fell', 'p.name': 'Eric Schaeffer'}, {'m.title': 'Burnt by the Sun (Utomlyonnye solntsem)', 'p.name': 'Nikita Mikhalkov'}, {'m.title': 'Forget Paris', 'p.name': 'Billy Crystal'}, {'m.title': 'Gumby: The Movie', 'p.name': 'Art Clokey'}, {'m.title': 'Hoop Dreams', 'p.name': 'Steve James'}, {'m.title': "Mary Shelley's Frankenstein (Frankenstein)", 'p.name': 'Kenneth Branagh'}]

> Finished chain.


{'query': 'Identify movies where directors also played a role in the film',
 'result': 'Mighty Aphrodite, Forget Paris,  "Mary Shelley\'s Frankenstein (Frankenstein)". \n'}

In [52]:
chain.invoke("How many movies has Tom Hanks acted in, tell me the number")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) 
RETURN count(m) AS movieCount

Full Context:
[{'movieCount': 2}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in, tell me the number',
 'result': "I don't know the answer. \n"}